In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]
# RESULTS_DIR = 'results_005_lang_id_classification'
RESULTS_DIR = 'results_005_lang_id_classification_dropout_0.2'
GPU_ID = 1

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
bias_category_specification = [
    {
        "category": "utterance",
        "subcategories": [
            {
                "subcategory": "verbal_nod",
                "tags": set(["utt-verbal-nod"])
            },
            {
                "subcategory": "multilingual",
                "tags": set(["utt-multi-lingual", "utt-multi-lingual-named-endity"])
            }
        ]
    },
    {
        "category": "speaker_count",
        "subcategories": [
            {
                "subcategory": "single",
                "tags": set(["spkr-single"])
            },
            {
                "subcategory": "multiple",
                "tags": set(["spkr-mult", "spkr-multi"])
            }
        ]
    },
    {
        "category": "gender",
        "subcategories": [
            {
                "subcategory": "male",
                "tags": set(["spkr-male"])
            },
            {
                "subcategory": "female",
                "tags": set(["spkr-female"])
            },
        ]
    },
    {
        "category": "language",
        "subcategories": [
            {
                "subcategory": "susu",
                "tags": set(["lng-susu"])
            },
            {
                "subcategory": "maninka",
                "tags": set(["lng-maninka"])
            },
            {
                "subcategory": "pular",
                "tags": set(["lng-pular"])
            }
        ]
    },
    {
        "category": "channel",
        "subcategories": [
            {
                "subcategory": "telephone",
                "tags": set(["ct-telephone"])
            },
            {
                "subcategory": "noise",
                "tags": set(["ct-noise"])
            },
            {
                "subcategory": "music",
                "tags": set(["ct-fg-music", "ct-tr-music", "ct-bg-music"])
            }
        ]
    }
]

flat_bias_category_specification = {}

for c in bias_category_specification:
    for sc in c['subcategories']:
        k = f"{c['category']}_{sc['subcategory']}"
        flat_bias_category_specification[k] = sc['tags']

_ = [print(f"{k}: {v}") for k,v in flat_bias_category_specification.items()]

utterance_verbal_nod: {'utt-verbal-nod'}
utterance_multilingual: {'utt-multi-lingual-named-endity', 'utt-multi-lingual'}
speaker_count_single: {'spkr-single'}
speaker_count_multiple: {'spkr-mult', 'spkr-multi'}
gender_male: {'spkr-male'}
gender_female: {'spkr-female'}
language_susu: {'lng-susu'}
language_maninka: {'lng-maninka'}
language_pular: {'lng-pular'}
channel_telephone: {'ct-telephone'}
channel_noise: {'ct-noise'}
channel_music: {'ct-tr-music', 'ct-bg-music', 'ct-fg-music'}


In [5]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [6]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label, tag_set
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels, audio_tags = zip(*data)

## Inspect label counts

In [7]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [8]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels, audio_tags = zip(*balanced_data)

In [9]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


## Inspect bias category counts in balanded data

In [10]:
def inspect_bias_category_counts():
    for name, tags in flat_bias_category_specification.items():
        count = len([ts for ts in audio_tags if len(tags.intersection(ts))>0])
        print(name, count, "/", len(audio_tags))
            
inspect_bias_category_counts()

utterance_verbal_nod 48 / 84
utterance_multilingual 21 / 84
speaker_count_single 25 / 84
speaker_count_multiple 58 / 84
gender_male 81 / 84
gender_female 15 / 84
language_susu 28 / 84
language_maninka 28 / 84
language_pular 28 / 84
channel_telephone 27 / 84
channel_noise 21 / 84
channel_music 24 / 84


# Prepare 10 cross validation folds

In [11]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(test_index_set)),
    }


# Load features

In [12]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [13]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [14]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features[feature_name][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [15]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [16]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [17]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [19]:
dropout_p = 0.2
class LangIdCNN_Std1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Std2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Std2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.std(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.std(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.std(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

class LangIdCNN_Mean1(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean1, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x
    
    
class LangIdCNN_Mean2(nn.Module):
    def __init__(self):
        super(LangIdCNN_Mean2, self).__init__()
        
        self.conv0 = nn.Conv1d(in_channels=512, out_channels=3, kernel_size=1)
        
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=dropout_p)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=dropout_p)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=dropout_p)
        self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop4 = nn.Dropout(p=dropout_p)
        self.pool4 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.lin5 = nn.Linear(in_features=9, out_features=3)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        
        x = self.conv0(x)
        
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        v1 = torch.mean(x, dim=2)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        v2 = torch.mean(x, dim=2)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.drop4(x)
        x = self.pool4(x)
        
        v3 = torch.mean(x, dim=2)
        
        v = torch.cat((v1, v2, v3), axis=1)
        
        x = self.lin5(v)
        
        return x

# Train Classification Models

In [20]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']    

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)
    train_tags = np.take(audio_tags, train_indices, axis=0)
    train_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        train_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in train_tags]

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    test_tags = np.take(audio_tags, test_indices, axis=0)
    test_bias_category_labels = {}
    for category, tags in flat_bias_category_specification.items():
        test_bias_category_labels[category] = [1 if len(tags.intersection(ts))>0 else 0 for ts in test_tags]
        
    
    return train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y, train_bias_category_labels, test_bias_category_labels = \
        get_data_for_fold(fold_id, feature_name, batch_size)
    
    
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader, train_bias_category_labels, test_bias_category_labels


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [21]:
def train_on_fold(model_class, fold_id, feature_name, batch_size, epochs):
    device = torch.device(f"cuda:{GPU_ID}")
    torch.manual_seed(0)
    results = {}
    
    model = model_class().to(device)

    train_loader, test_loader, train_bias_category_labels, test_bias_category_labels = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(model_class(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            x = x.to(device)
            y = y.to(device)
            
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )
            true_train_classes.extend(y.cpu().numpy())
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_n = len(true_train_classes)
        
        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)
        train_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes, sample_weight=sw)
            for category, sw in train_bias_category_labels.items()
        }
        


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs).cpu().numpy()
            )

            true_test_classes.extend(y.cpu().numpy())

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_n = len(true_test_classes)
        
        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)
        test_acc_by_bais_category = {
            category: sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes, sample_weight=sw)
            for category, sw in test_bias_category_labels.items()
        }

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'epoch': epoch,
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc,
            'train_n': train_n,
            'test_n': test_n,
            
        }
        
        for c in train_acc_by_bais_category:
            results[epoch][f"train_acc_{c}"] = train_acc_by_bais_category[c]
            results[epoch][f"train_n_{c}"] = int(np.sum(train_bias_category_labels[c]))
            
        for c in test_acc_by_bais_category:
            results[epoch][f"test_acc_{c}"] = test_acc_by_bais_category[c]
            results[epoch][f"test_n_{c}"] = int(np.sum(test_bias_category_labels[c]))
        
    del model
    return results
    

In [22]:
import csv
from pathlib import Path

def save_results(model_name, all_folds_results):
    for result_entry in all_folds_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        
        Path(RESULTS_DIR).mkdir(exist_ok=True)
        fname = f"{RESULTS_DIR}/{model_name}/{feature_name}_{fold_index}.csv"
        Path(fname).parent.mkdir(parents=True, exist_ok=True)
        with open(fname, 'w') as f:
            fieldnames = sorted(result_entry['epochs'][1].keys())
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            
            writer.writeheader()
            
            for epoch in sorted(result_entry['epochs'].keys()):

                writer.writerow(result_entry['epochs'][epoch])

In [ ]:
model_classes = [
    LangIdCNN_Std1,
    LangIdCNN_Mean1,
    LangIdCNN_Std2,
    LangIdCNN_Mean2
]

for model_class in model_classes:
    all_folds_results = []
    for fold_index in cv_folds:
                    
        for feature_name in raw_features:
            print(f"{model_class.__name__} using {feature_name} on fold#{fold_index}")
            resutls = train_on_fold(model_class, fold_index, feature_name, batch_size=100, epochs=1000)
            all_folds_results.append({
                'fold_index': fold_index,
                'feature_name': feature_name,
                'epochs': resutls
            })
            save_results(model_class.__name__, all_folds_results)

LangIdCNN_Std1 using wav2vec_features-c on fold#0
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       AvgPool1d-3       [10, 1, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              12              12
         Dropout-5       [10, 3, 1496]               0               0
       AvgPool1d-6        [10, 3, 748]               0               0
          Conv1d-7        [10, 3, 746]              30              30
         Dropout-8        [10, 3, 746]               0               0
       AvgPool1d-9        [10, 3, 373]               0               0
         Conv1d-10        [10, 3, 371]              30              30
        Dropout-11        [10, 3, 371]               0               0
      AvgPool1d-12        

Epoch: 830. Train Loss: 0.05247. Test Loss: 2.734. Train Acc: 1.0. Test Acc:0.5455
Epoch: 840. Train Loss: 0.05867. Test Loss: 2.781. Train Acc: 1.0. Test Acc:0.5455
Epoch: 850. Train Loss: 0.05916. Test Loss: 2.776. Train Acc: 1.0. Test Acc:0.5455
Epoch: 860. Train Loss: 0.0348. Test Loss: 2.8. Train Acc: 1.0. Test Acc:0.5455
Epoch: 870. Train Loss: 0.03965. Test Loss: 2.888. Train Acc: 1.0. Test Acc:0.5152
Epoch: 880. Train Loss: 0.05809. Test Loss: 2.939. Train Acc: 1.0. Test Acc:0.5152
Epoch: 890. Train Loss: 0.03983. Test Loss: 2.925. Train Acc: 1.0. Test Acc:0.5455
Epoch: 900. Train Loss: 0.04677. Test Loss: 2.884. Train Acc: 1.0. Test Acc:0.5455
Epoch: 910. Train Loss: 0.03988. Test Loss: 2.964. Train Acc: 1.0. Test Acc:0.5455
Epoch: 920. Train Loss: 0.03363. Test Loss: 2.996. Train Acc: 1.0. Test Acc:0.5455
Epoch: 930. Train Loss: 0.05697. Test Loss: 3.027. Train Acc: 1.0. Test Acc:0.5152
Epoch: 940. Train Loss: 0.04315. Test Loss: 3.01. Train Acc: 1.0. Test Acc:0.5455
Epoch: 9

Epoch: 640. Train Loss: 0.1716. Test Loss: 1.365. Train Acc: 0.9804. Test Acc:0.4848
Epoch: 650. Train Loss: 0.1735. Test Loss: 1.375. Train Acc: 0.9412. Test Acc:0.4848
Epoch: 660. Train Loss: 0.156. Test Loss: 1.412. Train Acc: 0.9608. Test Acc:0.4848
Epoch: 670. Train Loss: 0.1414. Test Loss: 1.403. Train Acc: 0.9804. Test Acc:0.4848
Epoch: 680. Train Loss: 0.1403. Test Loss: 1.413. Train Acc: 1.0. Test Acc:0.4848
Epoch: 690. Train Loss: 0.1427. Test Loss: 1.446. Train Acc: 1.0. Test Acc:0.4848
Epoch: 700. Train Loss: 0.1505. Test Loss: 1.433. Train Acc: 0.9804. Test Acc:0.5152
Epoch: 710. Train Loss: 0.1398. Test Loss: 1.453. Train Acc: 1.0. Test Acc:0.5152
Epoch: 720. Train Loss: 0.1286. Test Loss: 1.473. Train Acc: 0.9804. Test Acc:0.4848
Epoch: 730. Train Loss: 0.1227. Test Loss: 1.465. Train Acc: 1.0. Test Acc:0.5152
Epoch: 740. Train Loss: 0.1009. Test Loss: 1.503. Train Acc: 1.0. Test Acc:0.4848
Epoch: 750. Train Loss: 0.09615. Test Loss: 1.514. Train Acc: 1.0. Test Acc:0.515

Epoch: 460. Train Loss: 0.1175. Test Loss: 2.291. Train Acc: 1.0. Test Acc:0.6667
Epoch: 470. Train Loss: 0.1165. Test Loss: 2.327. Train Acc: 1.0. Test Acc:0.697
Epoch: 480. Train Loss: 0.1024. Test Loss: 2.432. Train Acc: 1.0. Test Acc:0.6667
Epoch: 490. Train Loss: 0.09404. Test Loss: 2.477. Train Acc: 1.0. Test Acc:0.697
Epoch: 500. Train Loss: 0.09548. Test Loss: 2.599. Train Acc: 1.0. Test Acc:0.6667
Epoch: 510. Train Loss: 0.09505. Test Loss: 2.7. Train Acc: 1.0. Test Acc:0.6667
Epoch: 520. Train Loss: 0.07574. Test Loss: 2.756. Train Acc: 1.0. Test Acc:0.697
Epoch: 530. Train Loss: 0.08328. Test Loss: 2.836. Train Acc: 1.0. Test Acc:0.697
Epoch: 540. Train Loss: 0.06998. Test Loss: 2.905. Train Acc: 1.0. Test Acc:0.697
Epoch: 550. Train Loss: 0.06427. Test Loss: 2.956. Train Acc: 1.0. Test Acc:0.697
Epoch: 560. Train Loss: 0.06361. Test Loss: 3.02. Train Acc: 1.0. Test Acc:0.697
Epoch: 570. Train Loss: 0.05692. Test Loss: 3.037. Train Acc: 1.0. Test Acc:0.697
Epoch: 580. Train 

Epoch: 280. Train Loss: 0.6396. Test Loss: 1.509. Train Acc: 0.6667. Test Acc:0.3636
Epoch: 290. Train Loss: 0.6227. Test Loss: 1.559. Train Acc: 0.7647. Test Acc:0.3636
Epoch: 300. Train Loss: 0.6191. Test Loss: 1.622. Train Acc: 0.6667. Test Acc:0.3636
Epoch: 310. Train Loss: 0.5965. Test Loss: 1.655. Train Acc: 0.7843. Test Acc:0.3636
Epoch: 320. Train Loss: 0.5861. Test Loss: 1.715. Train Acc: 0.7843. Test Acc:0.3636
Epoch: 330. Train Loss: 0.5666. Test Loss: 1.744. Train Acc: 0.7255. Test Acc:0.3636
Epoch: 340. Train Loss: 0.5596. Test Loss: 1.821. Train Acc: 0.7451. Test Acc:0.3636
Epoch: 350. Train Loss: 0.553. Test Loss: 1.888. Train Acc: 0.7843. Test Acc:0.3636
Epoch: 360. Train Loss: 0.5357. Test Loss: 1.923. Train Acc: 0.7843. Test Acc:0.3636
Epoch: 370. Train Loss: 0.5169. Test Loss: 1.928. Train Acc: 0.7843. Test Acc:0.3939
Epoch: 380. Train Loss: 0.5153. Test Loss: 1.972. Train Acc: 0.8039. Test Acc:0.3636
Epoch: 390. Train Loss: 0.516. Test Loss: 2.03. Train Acc: 0.7451.

...

Epoch: 410. Train Loss: 0.171. Test Loss: 0.9327. Train Acc: 0.9608. Test Acc:0.7576
Epoch: 420. Train Loss: 0.145. Test Loss: 0.9527. Train Acc: 0.9804. Test Acc:0.7576
Epoch: 430. Train Loss: 0.1252. Test Loss: 0.976. Train Acc: 1.0. Test Acc:0.7576
Epoch: 440. Train Loss: 0.0991. Test Loss: 1.001. Train Acc: 1.0. Test Acc:0.7576
Epoch: 450. Train Loss: 0.08742. Test Loss: 1.026. Train Acc: 1.0. Test Acc:0.7576
Epoch: 460. Train Loss: 0.07037. Test Loss: 1.061. Train Acc: 1.0. Test Acc:0.7576
Epoch: 470. Train Loss: 0.06272. Test Loss: 1.1. Train Acc: 1.0. Test Acc:0.7576
Epoch: 480. Train Loss: 0.0547. Test Loss: 1.118. Train Acc: 1.0. Test Acc:0.7273
Epoch: 490. Train Loss: 0.04724. Test Loss: 1.15. Train Acc: 1.0. Test Acc:0.7273
Epoch: 500. Train Loss: 0.03988. Test Loss: 1.19. Train Acc: 1.0. Test Acc:0.7273
Epoch: 510. Train Loss: 0.0356. Test Loss: 1.208. Train Acc: 1.0. Test Acc:0.697
Epoch: 520. Train Loss: 0.03142. Test Loss: 1.239. Train Acc: 1.0. Test Acc:0.7273
Epoch: 53

Epoch: 220. Train Loss: 0.4208. Test Loss: 1.321. Train Acc: 0.8627. Test Acc:0.5152
Epoch: 230. Train Loss: 0.3932. Test Loss: 1.386. Train Acc: 0.8824. Test Acc:0.5152
Epoch: 240. Train Loss: 0.3595. Test Loss: 1.47. Train Acc: 0.902. Test Acc:0.5152
Epoch: 250. Train Loss: 0.3181. Test Loss: 1.56. Train Acc: 0.9608. Test Acc:0.5152
Epoch: 260. Train Loss: 0.2818. Test Loss: 1.679. Train Acc: 0.9608. Test Acc:0.5455
Epoch: 270. Train Loss: 0.2541. Test Loss: 1.779. Train Acc: 0.9804. Test Acc:0.5455
Epoch: 280. Train Loss: 0.2223. Test Loss: 1.897. Train Acc: 0.9804. Test Acc:0.5455
Epoch: 290. Train Loss: 0.1886. Test Loss: 2.045. Train Acc: 1.0. Test Acc:0.5455
Epoch: 300. Train Loss: 0.1606. Test Loss: 2.161. Train Acc: 1.0. Test Acc:0.5758
Epoch: 310. Train Loss: 0.136. Test Loss: 2.284. Train Acc: 1.0. Test Acc:0.5758
Epoch: 320. Train Loss: 0.1209. Test Loss: 2.412. Train Acc: 1.0. Test Acc:0.5455
Epoch: 330. Train Loss: 0.1031. Test Loss: 2.522. Train Acc: 1.0. Test Acc:0.5455

Epoch: 30. Train Loss: 1.099. Test Loss: 1.103. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 40. Train Loss: 1.094. Test Loss: 1.103. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 50. Train Loss: 1.089. Test Loss: 1.099. Train Acc: 0.3333. Test Acc:0.3333
Epoch: 60. Train Loss: 1.083. Test Loss: 1.09. Train Acc: 0.3922. Test Acc:0.303
Epoch: 70. Train Loss: 1.075. Test Loss: 1.08. Train Acc: 0.4706. Test Acc:0.4545
Epoch: 80. Train Loss: 1.065. Test Loss: 1.065. Train Acc: 0.4314. Test Acc:0.4848
Epoch: 90. Train Loss: 1.051. Test Loss: 1.047. Train Acc: 0.4902. Test Acc:0.4545
Epoch: 100. Train Loss: 1.036. Test Loss: 1.024. Train Acc: 0.4902. Test Acc:0.4242
Epoch: 110. Train Loss: 1.016. Test Loss: 0.9988. Train Acc: 0.4902. Test Acc:0.4242
Epoch: 120. Train Loss: 0.9913. Test Loss: 0.9742. Train Acc: 0.4902. Test Acc:0.4545
Epoch: 130. Train Loss: 0.9656. Test Loss: 0.9509. Train Acc: 0.4902. Test Acc:0.4545
Epoch: 140. Train Loss: 0.9309. Test Loss: 0.9333. Train Acc: 0.5294. Test Acc:0.

LangIdCNN_Mean2 using retrained-wav2vec_features-c on fold#7
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 3, 2998]           1,539           1,539
          Conv1d-2       [10, 1, 2996]              10              10
         Dropout-3       [10, 1, 2996]               0               0
       AvgPool1d-4       [10, 1, 1498]               0               0
          Conv1d-5       [10, 3, 1496]              12              12
         Dropout-6       [10, 3, 1496]               0               0
       AvgPool1d-7        [10, 3, 748]               0               0
          Conv1d-8        [10, 3, 746]              30              30
         Dropout-9        [10, 3, 746]               0               0
      AvgPool1d-10        [10, 3, 373]               0               0
         Conv1d-11        [10, 3, 371]              30              30
        Dropout

Epoch: 810. Train Loss: 0.005743. Test Loss: 0.9485. Train Acc: 1.0. Test Acc:0.8788
Epoch: 820. Train Loss: 0.005046. Test Loss: 0.952. Train Acc: 1.0. Test Acc:0.8788
Epoch: 830. Train Loss: 0.004705. Test Loss: 0.9558. Train Acc: 1.0. Test Acc:0.8788
Epoch: 840. Train Loss: 0.004871. Test Loss: 0.963. Train Acc: 1.0. Test Acc:0.8788
Epoch: 850. Train Loss: 0.004474. Test Loss: 0.9671. Train Acc: 1.0. Test Acc:0.8788
Epoch: 860. Train Loss: 0.004304. Test Loss: 0.9712. Train Acc: 1.0. Test Acc:0.8788
Epoch: 870. Train Loss: 0.004028. Test Loss: 0.9766. Train Acc: 1.0. Test Acc:0.8788
Epoch: 880. Train Loss: 0.00413. Test Loss: 0.9835. Train Acc: 1.0. Test Acc:0.8788
Epoch: 890. Train Loss: 0.003622. Test Loss: 0.9924. Train Acc: 1.0. Test Acc:0.8788
Epoch: 900. Train Loss: 0.003388. Test Loss: 0.9933. Train Acc: 1.0. Test Acc:0.8788
Epoch: 910. Train Loss: 0.003345. Test Loss: 0.9978. Train Acc: 1.0. Test Acc:0.8788
Epoch: 920. Train Loss: 0.003143. Test Loss: 1.003. Train Acc: 1.0. 